In [46]:
import fasttext
from scipy import spatial
from sklearn import metrics

import os
import sys
import csv

from keras.layers import Input, Embedding, LSTM, Dense, Dot, Reshape, Flatten, Concatenate, Dropout
from keras.models import Model
import numpy as np

import tensorflow as tf
from sklearn.metrics import roc_auc_score
from collections import defaultdict

In [48]:
model = fasttext.load_model('../models/ru_tg_lenta_vector_model.bin')

In [26]:
print(spatial.distance.cosine(np.array(model.get_word_vector('tiger')), np.array(model.get_word_vector('elephant'))))

0.11214053630828857


In [49]:
import json
all_data = json.load(open('/Users/ilya-gusev/data/ru_tg_texts.json'))

In [50]:
all_data = sorted(all_data, key=lambda x:x['date'])

In [51]:
for row in all_data[0::2000]:
    print(row['date'])
    print(row['text'].replace('\n', ' '))


Новости и афишаДостопримечательностиПланы развития Участие в мероприятии только как зритель Фильм посвящен жизни и творческому пути гениального художника и скульптора эпохи Возрождения Микеланджело Буонарроти, за преданность которого борются два могущественных итальянских семейства. Андрей Кончаловский снимал картину в Италии — в Риме, Тоскане, а также в городе Каррара, где добывают каррарский мрамор, который Микеланджело использовал в своих работах. Узнать расписание сеансов можно на сайте сети кинотеатров «Москино». Жанр: драма, биография, история Режиссер: Андрей Кончаловский В ролях: Альберто Тестоне, Юлия Высоцкая, Орсо Мария Гуэррини, Глен Блэкхолл Год: 2019 Продолжительность: 2 часа 14 минут 
2019-11-01T09:29:10+00:00
В Telegram-каналах опубликовали видео с камер видеонаблюдения дома бывшего главы Киселёвска Сергея Лаврентьева. На кадрах видно, что бывший градоначальник бежит с ружьём за одним из злоумышленников. За ним следует второй пробравшийся на частную территорию мужчина.

In [52]:
last_host_end = {}

left = []
texts_left = []
right = []
texts_right = []
y = []

def words_to_embed(model, words):
    vectors = [model.get_word_vector(w) for w in words]
    norm_vectors = [x / np.linalg.norm(x) for x in vectors]
    avg_wv = np.mean(norm_vectors, axis=0)
    max_wv = np.max(norm_vectors, axis=0)
    min_wv = np.min(norm_vectors, axis=0)
    return np.concatenate((avg_wv, max_wv, min_wv))

for count, row in enumerate(all_data):
    if count % 1000 == 0:
        print(count)
    
    host = row['site_name']
    text = row['text'].strip().replace('\n', ' ')
    date = row['date']
    
    words = text.split()
    words = [w for w in words if w != '']
    if len(words) < 4:
        continue
    words = words[:300]
        
    border = len(words) // 2
    begin_words = words[:border]
    end_words = words[border:]
    left_sample = words_to_embed(model, begin_words)
    right_sample = words_to_embed(model, end_words)
    
    left.append(left_sample)
    texts_left.append(" ".join(begin_words))
    right.append(right_sample)
    texts_right.append(" ".join(end_words))
    y.append(1)
    if host in last_host_end:
        left.append(left_sample)
        texts_left.append(" ".join(begin_words))
        right.append(last_host_end[host][0])
        texts_right.append(last_host_end[host][1])
        y.append(0)
    last_host_end[host] = (right_sample, " ".join(end_words))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000


In [54]:
test_size = 30000
train_left = left[:-test_size ]
test_left = left[-test_size :]
train_right = right[:-test_size ]
test_right = right[-test_size :]
train_y = y[:-test_size ]
test_y = y[-test_size:]

texts_test_left = texts_left[-test_size:]
texts_test_right = texts_right[-test_size:]

In [55]:
scores = []
for i in range(0,len(test_y)):
    scores.append(-spatial.distance.cosine(test_left[i], test_right[i]))

In [56]:
metrics.roc_auc_score(test_y, scores)

0.7362494245329649

In [57]:
left_input = Input(shape=(150,), dtype='float32')
right_input = Input(shape=(150,), dtype='float32')
dense = Dense(50, activation='linear')
left_dense = dense(left_input)
right_dense = dense(right_input)
dot_layer = Dense(1, activation='sigmoid')(Dot(axes=1, normalize=True)([left_dense, right_dense]))
nn_model = Model(inputs=[left_input, right_input], output=dot_layer)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  import sys


In [58]:
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [59]:
from keras.callbacks.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
nn_model.fit([np.array(train_left), np.array(train_right)],
             np.array(train_y),
             batch_size=256,
             epochs=100,
             callbacks=[es,],
             validation_data=([np.array(test_left), np.array(test_right)], np.array(test_y)),
             verbose=2)

Train on 208200 samples, validate on 30000 samples
Epoch 1/100
 - 4s - loss: 0.6235 - accuracy: 0.7445 - val_loss: 0.5818 - val_accuracy: 0.7705
Epoch 2/100
 - 3s - loss: 0.5309 - accuracy: 0.8087 - val_loss: 0.5127 - val_accuracy: 0.8036
Epoch 3/100
 - 3s - loss: 0.4669 - accuracy: 0.8386 - val_loss: 0.4637 - val_accuracy: 0.8279
Epoch 4/100
 - 3s - loss: 0.4187 - accuracy: 0.8580 - val_loss: 0.4220 - val_accuracy: 0.8460
Epoch 5/100
 - 3s - loss: 0.3823 - accuracy: 0.8708 - val_loss: 0.3984 - val_accuracy: 0.8497
Epoch 6/100
 - 3s - loss: 0.3536 - accuracy: 0.8785 - val_loss: 0.3680 - val_accuracy: 0.8633
Epoch 7/100
 - 3s - loss: 0.3309 - accuracy: 0.8849 - val_loss: 0.3493 - val_accuracy: 0.8687
Epoch 8/100
 - 3s - loss: 0.3129 - accuracy: 0.8892 - val_loss: 0.3383 - val_accuracy: 0.8693
Epoch 9/100
 - 3s - loss: 0.2984 - accuracy: 0.8928 - val_loss: 0.3238 - val_accuracy: 0.8765
Epoch 10/100
 - 3s - loss: 0.2867 - accuracy: 0.8957 - val_loss: 0.3210 - val_accuracy: 0.8752
Epoch 11

In [60]:
pred = nn_model.predict([np.array(test_left), np.array(test_right)])

In [61]:
nn_scores = [float(pred[i]) for i in range(0,len(pred))]

In [62]:
print(len(nn_scores), len(test_y), len(texts_left), len(texts_right))
for i in range(0,500):
    print("===============")
    print(nn_scores[i], test_y[i])
    print(texts_test_left[i])
    print("@@@")
    print(texts_test_right[i])
    print("")

30000 30000 238200 238200
0.9710039496421814 1
Лидер общественно-политического движения "Новая Россия" Никита Исаев умер в возрасте 41 года, сообщает "Интерфакс" со ссылкой на несколько источников в "Новой России". О смерти Исаева также написала в фейсбуке его помощница Алина Жестовская. Добавлено. Представитель Московского межрегионального следственного управления на транспорте СК Елена Марковская сообщила о начале проверки по факту Исаева, передает ТАСС. "По предварительным данным, смерть не носит криминальный характер", — сказала она. Исаеву стало плохо в поезде
@@@
Тамбов — Москва, у него была сердечно-сосудистая недостаточность и произошла остановка сердца, сообщает "Комсомольская правда", где политик вел колонку. Официально причина смерти пока не объявлена. Исаев был лидером "Новой России" с января 2017 года. До этого в разные годы он работал в Министерстве регионального развития по вопросам Северного Кавказа, Южной Осетии и Абхазии и в общественном совете Госдумы. До 2016 года И

0.001991361379623413 0
По словам Олега Ещенко, он неоднократно разговаривал с дочерью о том, что у этих отношений нет будущего Краснодар, 16 ноября. После похорон Анастасии Ещенко, которые состоялись сегодня в Краснодарском крае, отец погибшей девушки заявил, что она любила предполагаемого убийцу Олега Соколова. По словам мужчины, он предупреждал дочку, что у этих отношений не будет будущего. - Я рассуждал так, что если ей хорошо с ним, значит и
@@@
врачам, - сообщили РИА «Регион online» очевидцы произошедшего. Затем неравнодушный человек вызвал скорую помощь. Медики ехали к месту инцидента около семи минут. Все это время активист оказывал первую помощь своему подопечному. Затем мужчину унесли на носилках к карете скорой помощи и увезли в больницу. Ранее портал РИА «Регион online» сообщил, что жители Сочи обсуждают детали убийства женщины. Жители всю ночь не могли заснуть из-за душераздирающих криков под окнами. Читайте нас в News

0.8775289058685303 1
Профильный департамент культуры и


0.5177361369132996 0
Бывший форвард киевского «Динамо» и национальной сборной Украины Артем Милевский, ныне выступающий за белорусский «Динамо-Брест», прокомментировал расистский скандал, произошедший на матче 14-го тура УПЛ «Шахтер» — «Динамо» (1:0). «Фанаты «Динамо» имитировали звуки обезьян во время матча с «Шахтером»? Так же всю жизнь было. У нас [в «Динамо-Брест»] черный парнишка есть, Кики. Я
@@@
дрогнул при ударе ногой в исполнении юной тхэквондистки. Напомним, Ибрагимович провел за «Гэлакси» 53 матча, где забил 52 мяча и отдал 17 результативных передач. Ранее сообщалось, что Ибрагимович может продолжить карьеру в Серии А. Источник — Sport.ua

0.972572922706604 1
Кабинет министр одобрил для внесения в Государственную думу законопроект о единых денежных выплатах военнослужащим по призыву в размере 2 тысяч рублей. В эти выплаты входят оклад, ежемесячные надбавки за выполнение задач, связанных с риском для жизни и здоровья, за руководство воинским подразделением и работу с предста

In [63]:
metrics.roc_auc_score(test_y, nn_scores)

0.9559822629397594

In [64]:
import random
for _ in range(10000):
    i = int(random.random()*len(texts_test_left))
    j = int(random.random()*len(texts_test_right))
    pred = nn_model.predict([np.array([test_left[i]]), np.array([test_right[j]])])
    if float(pred[0]) > 0.9:
        print('======')
        print(pred)
        print(texts_test_left[i])
        print("@@@")
        print(texts_test_right[j])

[[0.9629695]]
Sport.ru/Лыжные гонки. Россиянка Наталья Непряева одержала победу в десятикилометровой гонке свободным стилем на лыжном турнире в Муонио (Финляндия).Норицын считает, что в биатлон из лыжных гонок можно пригласить только НепряевуНепряева преодолела дистанцию за 26 минут 35,2 секунды. Второй стала финка Кертту Нисканен (+15,0). Тройку лидеров замкнула представительница Германии Виктория Карл (+10,9). Лидия Дуркина стала шестой (+1:10,3), Анна Жеребятьева замкнула топ-10
@@@
прыжка. В итоге по сумме двух прокатов она набрала 234,47 балла — 74,21 за короткую и 160,26 за произвольную программы, что позволило ей занять первое место. Таким образом, благодаря победам на этапах в США и России Трусова пробилась в финал серии Гран-при, который пройдёт в Турине с 5 по 8 декабря. Ранее сообщалось, что Илья Авербух объяснил ошибки Трусовой на этапе Гран-при в Москве.
[[0.9601176]]
Недостаток сна является куда более распространенной причиной болей в шее, чем неправильная поза, сообщает 

[[0.9098155]]
«Салават Юлаев» в гостях крупно уступил ярославскому «Локомотиву», прервав серию из трёх побед подряд. В первом периоде на 11 минуте счёт открыл нападающий хозяев Степан Санников. Преимущество ярославцев увеличил Николай Коваленко. Во втором отрезке игры Тему Хартикайнен с передачи Умарка и
@@@
за 100 миллионов евро. Напомним, в текущем сезоне Антон провел девять матчей в составе «Локомотива» в РПЛ, забил два гола и сделал результативный пас. Алексей принял участие в 13 играх, записал на свой счет четыре гола и три передачи.
[[0.94357425]]
Новгородские спасатели продолжают вести восстановительные работы в подтопленных районах. По состоянию на 18:00 так и не освободились от воды 4 жилых дома. В зоне подтопления по-прежнему территории девяти районов. Три участка дорог восстановлены, 21 – закрыт для движения. Требуют ремонта 7 переправ. От территориальной РСЧС в спасательных мероприятиях задействовано 515 человек и
@@@
«После завершения ремонта газовики проверили герметичнос

[[0.98063755]]
Захваченные Россией украинские военные катера "Бердянск" и "Никополь", а также буксир "Яны Капу", вышли из акватории "Генмола" в Керчи в сопровождении российской береговой охраны. 18 ноября они будут переданы Украине, подтвердили в пресс-службе погрануправления ФСБ России по Республике Крым. "В соответствии с достигнутыми с украинской стороной договоренностями, три украинских судна - буксир "Яны Капу" и малые бронированные артиллерийские катера "Никополь" и "Бердянск" - буксируются из порта Керчь к согласованному месту передачи украинской стороне, которая состоится 18 ноября", - сказали в пресс-службе. Где именно состоится передача кораблей украинской стороне, в ФСБ не уточнили. Там воздержались от любых подробностей, касающихся процедуры передачи. Первым причал покинул "Яны Капу", затем на буксир был взят "Бердянск" с отремонтированной пробоиной на борту, которая была получена при захвате год назад. Защита украинских моряков намерена добиваться прекращения уголовного де

[[0.90761864]]
В субботу, 16 ноября, состоялись поединки девятого тура отборочного этапа к Евро-2020 в группе C. Сборная Германии без особых неожиданностей крупно обыграла Беларусь, а Нидерланды не сумели одолеть Северную Ирландию. Как бы там ни было, но Германия и Нидерланды оформили
@@@
24-х, которые будут играть на Евро-2020. Cписок сборных, успешно прошедших отбор, сейчас выглядит таким образом: от группы "А" - Англия и Чехия, от группы "В" - Украина и Португалия, от группы "С" - Германия и Нидерланды, от группы "Е" - Хорватия, от группы "F" - Испания и Швеция, от группы "G" - Польша и Австрия, от группы "Н" - Франция и Турция, от группы "I" - Бельгия и Россия, от группы "J" - Италия и Финляндия.
[[0.9655249]]
Недавно 12-летний сын Дмитрия Певцова в гневе ударил кулаком в стену и сломал себе руку. Это натолкнуло артиста на некоторые мысли, с которыми он поделился с подписчиками. Так, Дмитрий Певцов считает, что даже объективная ненависть способна лишь уничтожить человека, прежде вс

[[0.95542246]]
Сборная России по футболу 16 ноября уступила национальной команде Бельгии со счетом 1:4 в матче отборочного турнира чемпионата Европы – 2020. Встреча проходила на стадионе «Газпром Арена» в Санкт-Петербурге. Счет в матче на 19-й минуте открыл полузащитник бельгийцев Торган Азар. На 33-й минуте его старший брат Эден забил второй гол, а через семь минут он оформил дубль. Во втором тайме на 72-й минуте автором очередного гола в ворота россиян
@@@
хавбеки – воспитанники «Локомотива» и выступают за основную команду с 2013 года. В нынешнем сезоне Алексей в 13 матчах РПЛ забил 4 мяча (2 – с пенальти), а также отдал 2 результативные передачи. У Антона 2 мяча и 1 пас в 9 встречах.
[[0.90218925]]
РИАМО - 17 ноя. Деревянное здание загорелось в Новопетровском проезде столицы, огонь ликвидировали на площади порядка 20 квадратных метров, говорится в сообщении пресс-службы Главного управления МЧС России по Москве. «16 ноября 2019 года в 23:16 поступило сообщение о пожаре по адресу: Нов

In [65]:
embedder = Model(inputs=[left_input, ], output=left_dense)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [66]:
print(texts_test_left[0])
print(test_left[:1])
print(embedder.predict(np.array(test_left[:1])))

Лидер общественно-политического движения "Новая Россия" Никита Исаев умер в возрасте 41 года, сообщает "Интерфакс" со ссылкой на несколько источников в "Новой России". О смерти Исаева также написала в фейсбуке его помощница Алина Жестовская. Добавлено. Представитель Московского межрегионального следственного управления на транспорте СК Елена Марковская сообщила о начале проверки по факту Исаева, передает ТАСС. "По предварительным данным, смерть не носит криминальный характер", — сказала она. Исаеву стало плохо в поезде
[array([-0.02706261, -0.11791315,  0.05046882, -0.08823448,  0.01097538,
        0.00132969,  0.04315061, -0.09224065,  0.06618552, -0.13176964,
        0.06456982, -0.12267913, -0.0007471 , -0.09193623,  0.01136862,
        0.05678104,  0.02015842,  0.00426692,  0.09768451,  0.20502503,
       -0.06854028, -0.04092625, -0.07040061,  0.04940726, -0.01357833,
       -0.07360607,  0.13587156,  0.02732933, -0.07031792,  0.0651442 ,
        0.00371204, -0.06810868, -0.006746

In [67]:
matrix = dense.get_weights()[0]
bias = dense.get_weights()[1]

In [68]:
with open("matrix.txt", "w") as w:
    for row_num in range(matrix.shape[1]):
        row = []
        for col_num in range(matrix.shape[0]):
            row.append(float(matrix[col_num][row_num]))
        w.write(",".join(map(str, row)) + "\n")

with open("bias.txt", "w") as w:
    for value in bias:
        w.write("{}\n".format(value))

In [69]:
matrix

array([[ 0.9012191 ,  0.68869233, -0.74790806, ...,  0.3680066 ,
         0.2229627 , -0.8451709 ],
       [-0.06855872, -0.57508993,  1.2699703 , ..., -0.5286815 ,
         0.9098699 ,  0.13198073],
       [ 1.4904196 ,  0.22335042,  0.04915512, ...,  0.70552576,
         0.7166109 , -0.3658475 ],
       ...,
       [-0.12743777,  0.1915168 , -0.10990631, ...,  0.07598341,
        -0.09589029, -0.03054393],
       [-0.086417  ,  0.11990174,  0.14829984, ..., -0.01570303,
        -0.02618517, -0.00173675],
       [-0.07120676,  0.06567481,  0.00725528, ...,  0.18066365,
        -0.06922477,  0.00463527]], dtype=float32)